In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# Obtain github paths for data downloads
examples_url = "https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/shopping_queries_dataset_examples.parquet"
products_url = "https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/shopping_queries_dataset_products.parquet"

# Download examples with query-product mappings
examples_response = requests.get(examples_url)
with open("../data/raw/examples.parquet", "wb") as f:
    f.write(examples_response.content)
del examples_response # Free up memory before next download

# Download products with product details  
products_response = requests.get(products_url)
with open("../data/raw/products.parquet", "wb") as f:
    f.write(products_response.content)
del products_response # Free up memory from large download

In [2]:
# Load datasets
df_examples = pd.read_parquet("../data/raw/examples.parquet")
df_products = pd.read_parquet("../data/raw/products.parquet")

# Merge on product_locale and product_id, per github repo instructions
df_examples_products = pd.merge(
    df_examples, 
    df_products,
    how='left',
    left_on=['product_locale', 'product_id'],
    right_on=['product_locale', 'product_id']
)

# Preliminary look at the merged dataset
print(f"Dataset shape: {df_examples_products.shape}")
df_examples_products.head()

Dataset shape: (2621288, 14)


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,None,WhisperCeiling fans feature a totally enclosed...,Panasonic,White
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train,Homewerks 7141-80 Bathroom Fan Integrated LED ...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,80 CFM
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,None,OUTSTANDING PERFORMANCE: This Homewerk's bath ...,Homewerks,White
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train,Delta Electronics RAD80L BreezRadiance 80 CFM ...,This pre-owned or refurbished product has been...,Quiet operation at 1.5 sones\nBuilt-in thermos...,DELTA ELECTRONICS (AMERICAS) LTD.,White
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train,Panasonic FV-08VRE2 Ventilation Fan with Reces...,None,The design solution for Fan/light combinations...,Panasonic,White


In [3]:
# Filter for task 1 and target criteria
df_task1 = df_examples_products[df_examples_products["small_version"] == 1]

# Look at the down-sized small version dataset
print(f"Dataset small_version shape: {df_task1.shape}")

# Apply filters for US locale and Exact matches only
df_filtered = df_task1[
    (df_task1['product_locale'] == 'us') & 
    (df_task1['esci_label'] == 'E')
]

# Look at the further filtered dataset
print(f"Dataset filtered shape: {df_filtered.shape}")
print(f"Unique queries: {df_filtered['query'].nunique()}")

# Another quick look at the data
df_filtered.head()

Dataset small_version shape: (1118011, 14)
Dataset filtered shape: (261527, 14)
Unique queries: 29843


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
17,17,!awnmower tires without rims,1,B08L3B9B9P,us,E,1,1,train,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,MaxAuto 2-Pack 13x5.00-6 2PLY Turf Mower Tract...,Please check your existing tire Sidewall for t...,MaxAuto,None
20,20,!awnmower tires without rims,1,B07C1WZG12,us,E,1,1,train,(Set of 2) 15x6.00-6 Husqvarna/Poulan Tire Whe...,No fuss. Just take off your old assembly and r...,Tire size:15x6.00-6 Ply: 4 Tubeless\n6x4.5 Whe...,Antego Tire & Wheel,Husqvarna Silver
21,21,!awnmower tires without rims,1,B077QMNXTS,us,E,1,1,train,MaxAuto 2 Pcs 16x6.50-8 Lawn Mower Tire for Ga...,<br>Tire Specifications:<br> 1. Material: Rubb...,"Set of 2 16X6.50-8, 16x6.50x8, 16-6.50-8 Lawn ...",MaxAuto,Black
23,23,!awnmower tires without rims,1,B06XX6BM2R,us,E,1,1,train,"MARASTAR 21446-2PK 15x6.00-6"" Front Tire Assem...",None,Tire: 2 pack 15x6. 00-6 tube-type turf SAVER t...,MARASTAR,None
26,26,!awnmower tires without rims,1,B0089RNSNM,us,E,1,1,train,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Honda 42710-VE2-M02ZE (Replaces 42710-VE2-M01Z...,Set of 2 Honda OEM Rear Wheels\nReplaces 42710...,Honda,None


* Notice '!' exclamation point instead of 'l' for the first letter in lawn mower for query col.
  * Some text cleanup likely

In [4]:
# Further look at the data before down-sampling
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 261527 entries, 17 to 2618569
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   example_id            261527 non-null  int64 
 1   query                 261527 non-null  object
 2   query_id              261527 non-null  int64 
 3   product_id            261527 non-null  object
 4   product_locale        261527 non-null  object
 5   esci_label            261527 non-null  object
 6   small_version         261527 non-null  int64 
 7   large_version         261527 non-null  int64 
 8   split                 261527 non-null  object
 9   product_title         261527 non-null  object
 10  product_description   134067 non-null  object
 11  product_bullet_point  231366 non-null  object
 12  product_brand         249806 non-null  object
 13  product_color         184355 non-null  object
dtypes: int64(4), object(10)
memory usage: 29.9+ MB


In [9]:
# Save current filtered dataset
df_filtered.to_parquet("../data/processed/df_filtered.parquet")

* Many missing values for the 'product_' related columns:
  * ~51% for 'product_description' are missing!
  * ~11% for 'product_bullet_point' are missing
  * ~4% for 'product_brand' are missing
  * ~29% for 'product_color' are missing
* I find it unlikely that Grainger would have such missing information for their product catalog:
  * Assumption to drop null values before creating ~500 row / 50 unique query sample dataset

In [5]:
# Drop null values for the 'product_' related columns
df_dropped = df_filtered.dropna(subset=['product_description', 'product_bullet_point', 'product_brand', 'product_color']).reset_index(drop=True)

# Look at the data again before trying down-sampling
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93296 entries, 0 to 93295
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   example_id            93296 non-null  int64 
 1   query                 93296 non-null  object
 2   query_id              93296 non-null  int64 
 3   product_id            93296 non-null  object
 4   product_locale        93296 non-null  object
 5   esci_label            93296 non-null  object
 6   small_version         93296 non-null  int64 
 7   large_version         93296 non-null  int64 
 8   split                 93296 non-null  object
 9   product_title         93296 non-null  object
 10  product_description   93296 non-null  object
 11  product_bullet_point  93296 non-null  object
 12  product_brand         93296 non-null  object
 13  product_color         93296 non-null  object
dtypes: int64(4), object(10)
memory usage: 10.0+ MB


* Full dataset with no missing values has ~93k records:
  * Try ~500 row / 50 unique query sample dataset creation

In [ ]:
# Create sample dataset: ~500 rows with ~50 unique queries
np.random.seed(2025) # set random seed for reproducibility (like to use the year)
unique_queries = df_dropped['query'].unique()

# Sample 50 queries randomly
n_queries = min(50, len(unique_queries))
sample_queries = np.random.choice(unique_queries, size=n_queries, replace=False)

# Get all products for these queries
df_sample = df_dropped[df_dropped['query'].isin(sample_queries)].copy()

# If too many rows, sample proportionally per query
if df_sample.shape[0] > 500:
    target_per_query = max(1, 500 // n_queries)
    df_sample = df_sample.groupby('query').apply(
        lambda x: x.sample(n=min(len(x), target_per_query), random_state=2025)
    ).reset_index(drop=True)
else:
    df_sample = df_sample.reset_index(drop=True)

print(f"Final sample: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")

# Look at the sample dataset head
# df_sample[['query', 'product_title']].head()
df_sample.head()

Final sample: 210 rows, 50 queries


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
0,117965,6 acrylic brushes for nails,4839,B087CJ6ZVV,us,E,1,1,test,"Yolife Acrylic Nail Art Brush, 100% Pure Kolin...",<b>Package Content: </b><p>1* Red Wooden Nail ...,❤️【Ingenious Design】100% Pure Kolinsky Sable N...,Yolife,Wood
1,117966,6 acrylic brushes for nails,4839,B083ZMG2ZX,us,E,1,1,test,Pana USA Acrylic Nail Brush100% Pure Kolinsky ...,PANA Finest Kolinsky Hair Professional Acrylic...,[Design] Size 6 Brush Handle | A smooth and st...,PANA,Mahogany
2,117967,6 acrylic brushes for nails,4839,B07Y55F8HH,us,E,1,1,test,1PCS Eval 100% Kolinsky Sable hair Brushes Acr...,Eval Crimped Shaped Kolinsky Sable Brushes #6 ...,★HIGH QUALITY HAIR:Brushes made of Kolinsky sa...,Eval,Purple
3,137801,a 6 motels,5780,B07MTMP7D5,us,E,1,1,train,"Evocel Galaxy A6 (2018) Case, [Explorer Series...",The Evocel Explorer Series Pro is a phone case...,"[Dependable Protection] Heavy duty, dual layer...",Evocel,Blue
4,137803,a 6 motels,5780,B0796M8RPJ,us,E,1,1,train,Aunifun Replacement Filter & Side Brush Kit fo...,<b>Fit for</b> ILIFE A6 A4 A4s Robot Vacuum Cl...,★Fit for ILIFE A6 A4 A4s Robot Vacuum Cleaner....,Aunifun,Replacement Filter & Side Brush Kit for ILIFE ...


* Only able to sample to 210 rows from non-null ~93k dataset:
  * May need to accept some missing 'product_' values and develop imputation strategy
  * Turn down-sampling into defined function and try different ways to handle missing values

In [ ]:
# Save the sample dataset
df_sample.to_parquet("semantic_search_dataset.parquet")
df_sample.to_csv("semantic_search_dataset.csv", index=False)

print(f"Dataset saved: {df_sample.shape[0]} rows, {df_sample['query'].nunique()} queries")
print("Files: semantic_search_dataset.parquet, semantic_search_dataset.csv")

In [ ]:
# Quick exploration of the final dataset
print("Dataset columns:")
print(df_sample.columns.tolist())
print(f"\nExample queries:")
for query in df_sample['query'].unique()[:3]:
    print(f"- {query}")